## Python Implementation

This example considers two supermarkets and nine warehouse candidates. The coordinates of each supermarket are provided in the following table.

| <i></i> | Coordinates |  
| --- | --- | 
| Supermarket 1 | (0,1.5) |
| Supermarket 2 | (2.5,1.2) |

The following table shows the coordinates of the candidate warehouse sites and the fixed cost of building the warehouse in millions of GBP.

| <i></i> | coordinates | fixed cost |
| --- | --- |  --- |
| Warehouse 1 | (0,0) | 3 |
| Warehouse 2 | (0,1) | 2 |
| Warehouse 3 | (0,2) | 3 |
| Warehouse 4 | (1,0) | 1 |
| Warehouse 5 | (1,1) | 3 | 
| Warehouse 6 | (1,2) | 3 |
| Warehouse 7 | (2,0) | 4 |
| Warehouse 8 | (2,1) | 3 |  
| Warehouse 9 | (2,2) | 2 |


The cost per mile is one million GBP.

## Python Implementation

We now import the Gurobi Python Module and other Python libraries. Then, we initialize the data structures with the given data.

In [1]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

In [2]:
supermarkets = {"supermarket1":(0, 1.5), "supermarket2":(2.5, 1.2)}
warehouses = {
    "warehouse1":(0.00,0.00),
    "warehouse2":(0.00,1.00),
    "warehouse3":(0.00,2.00),
    "warehouse4":(1.00,0.00),
    "warehouse5":(1.00,1.00),
    "warehouse6":(1.00,2.00),
    "warehouse7":(2.00,0.00),
    "warehouse8":(2.00,1.00),
    "warehouse9":(2.00,2.00)
}
warehouse_cost = {
    "warehouse1":3,
    "warehouse2":2,
    "warehouse3":3,
    "warehouse4":1,
    "warehouse5":3,
    "warehouse6":3,
    "warehouse7":4,
    "warehouse8":3,
    "warehouse9":2
}

# assume cost per mile is $1
cost_per_mile = 1

In [118]:
# Parameters
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

In [119]:
def compute_distance(loc1, loc2):
    dx = loc1[0]-loc2[0]
    dy = loc1[1]-loc2[1]
    return sqrt(dx**2+dy**2)

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
print(cartesian_prod)
# The product is not limited to numeric values; it can be used with any iterable e.g. strings, tuples, lists.
# The product returns all possible combinations of the elements in tuples.

shipping_cost = {(c,f): compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}
print(shipping_cost)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8)]
{(0, 0): 1.5, (0, 1): 0.5, (0, 2): 0.5, (0, 3): 1.8027756377319946, (0, 4): 1.118033988749895, (0, 5): 1.118033988749895, (0, 6): 2.5, (0, 7): 2.0615528128088303, (0, 8): 2.0615528128088303, (1, 0): 2.773084924772409, (1, 1): 2.5079872407968904, (1, 2): 2.6248809496813377, (1, 3): 1.9209372712298547, (1, 4): 1.5132745950421556, (1, 5): 1.7, (1, 6): 1.3, (1, 7): 0.5385164807134504, (1, 8): 0.9433981132056605}


### Decision Variable

In [120]:
model = gp.Model('facility_location')

select = model.addVars(num_facilities, vtype=GRB.BINARY, name="Select")
assign = model.addVars(num_customers, num_facilities, vtype=GRB.CONTINUOUS, lb=0.00, ub=1, name="Assign")
print(select)
print(assign)

{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>, 2: <gurobi.Var *Awaiting Model Update*>, 3: <gurobi.Var *Awaiting Model Update*>, 4: <gurobi.Var *Awaiting Model Update*>, 5: <gurobi.Var *Awaiting Model Update*>, 6: <gurobi.Var *Awaiting Model Update*>, 7: <gurobi.Var *Awaiting Model Update*>, 8: <gurobi.Var *Awaiting Model Update*>}
{(0, 0): <gurobi.Var *Awaiting Model Update*>, (0, 1): <gurobi.Var *Awaiting Model Update*>, (0, 2): <gurobi.Var *Awaiting Model Update*>, (0, 3): <gurobi.Var *Awaiting Model Update*>, (0, 4): <gurobi.Var *Awaiting Model Update*>, (0, 5): <gurobi.Var *Awaiting Model Update*>, (0, 6): <gurobi.Var *Awaiting Model Update*>, (0, 7): <gurobi.Var *Awaiting Model Update*>, (0, 8): <gurobi.Var *Awaiting Model Update*>, (1, 0): <gurobi.Var *Awaiting Model Update*>, (1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>, (1, 4): <gurobi.Var *Awaiting M

### Objective function

In [121]:
model.setObjective(
    gp.quicksum(setup_cost[f]*select[f]+shipping_cost[c,f]*assign[c,f]
                for f in range(num_facilities) for c in range(num_customers)), GRB.MINIMIZE)
# above is equal to
model.setObjective(select.prod(setup_cost)+assign.prod(shipping_cost), GRB.MINIMIZE)

### Constraint

In [122]:
#################### why this does not work? the sum of supply for one facility should add up to 1
model.addConstrs((gp.quicksum(assign[c,f] for c in range(num_customers)) <= select[f] for f in range(num_facilities)), name='Supply')
# demand
model.addConstrs((gp.quicksum(assign[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')
# for all 0<=assign<=1
model.addConstrs((assign[(c,f)] <= select[f] for c,f in cartesian_prod), name='Supply')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>}

In [123]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 29 rows, 27 columns and 81 nonzeros
Model fingerprint: 0x9e15f521
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 18 rows and 0 columns
Presolve time: 0.00s
Presolved: 11 rows, 27 columns, 45 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Found heuristic solution: objective 11.0385165

Root relaxation: objective 8.420937e+00, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0  

In [125]:
model.printAttr('X')


    Variable            X 
-------------------------
   Select[1]            1 
   Select[3]            1 
 Assign[0,1]            1 
 Assign[1,3]            1 
